In [389]:
import pandas as pd
import numpy as np

In [390]:
training_org=pd.read_csv("D:/CLASS/Term 2/Kaggle competition/city-bikes-demand/train.csv")

In [391]:
test_org=pd.read_csv("D:/CLASS/Term 2/Kaggle competition/city-bikes-demand/test.csv")

In [392]:
holiday_org=pd.read_csv("D:/CLASS/Term 2/Kaggle competition/city-bikes-demand/holidays.csv")

In [393]:
training_org.isnull().sum()

datetime                   0
summary                   26
icon                      26
precipIntensity         3725
precipProbability       3725
temperature               31
apparentTemperature       31
dewPoint                  27
humidity                  31
windSpeed                 44
windGust                3139
windBearing               55
cloudCover              1036
uvIndex                  566
visibility                55
precipType             14104
pressure                3766
precipAccumulation     22440
ozone                  15829
countRents                 0
dtype: int64

In [394]:
test_org.isnull().sum()

datetime                 0
summary                  0
icon                     0
precipIntensity          0
precipProbability        0
temperature              0
apparentTemperature      0
dewPoint                 0
humidity                 0
windSpeed                0
windGust                 0
windBearing              0
cloudCover               0
uvIndex                  0
visibility               0
precipType             381
pressure                 0
precipAccumulation     744
ozone                    0
dtype: int64

In [395]:
holiday_org.isnull().sum()

date       0
holiday    0
dtype: int64

In [396]:
training_org.shape

(22488, 20)

# Imputation

## deleting rows for some variables

In [397]:
training_org=training_org.dropna(subset=['summary', 'icon','temperature','apparentTemperature','dewPoint','humidity','windSpeed','windBearing','visibility'],how='any')

### precipIntensity

In [398]:
training_org.loc[training_org["precipIntensity"].isnull(),'summary'].unique()

array(['Clear', 'Partly Cloudy', 'Overcast', 'Mostly Cloudy', 'Foggy',
       'Windy and Mostly Cloudy', 'Windy and Partly Cloudy'], dtype=object)

In [399]:
training_org["precipIntensity"]=training_org["precipIntensity"].fillna(0)

### precipProbability

In [400]:
training_org.loc[training_org["precipProbability"].isnull(),'summary'].unique()

array(['Clear', 'Partly Cloudy', 'Overcast', 'Mostly Cloudy', 'Foggy',
       'Windy and Mostly Cloudy', 'Windy and Partly Cloudy'], dtype=object)

In [401]:
training_org["precipProbability"]=training_org["precipProbability"].fillna(0)

### precipType

In [402]:
training_org.loc[training_org["precipType"].isnull(),'summary'].unique()

array(['Mostly Cloudy', 'Overcast', 'Partly Cloudy', 'Clear', 'Foggy',
       'Windy and Partly Cloudy', 'Windy and Mostly Cloudy', 'Windy'],
      dtype=object)

In [403]:
test_org.loc[test_org["precipType"].isnull(),'summary'].unique()

array(['Mostly Cloudy', 'Overcast', 'Partly Cloudy', 'Clear'],
      dtype=object)

In [404]:
training_org["precipType"]=training_org["precipType"].fillna("non")

In [405]:
test_org["precipType"]=test_org["precipType"].fillna("non")

### windGust

In [406]:
from sklearn.linear_model import LinearRegression

In [407]:
lr=LinearRegression()

In [408]:
lr.fit(X=pd.DataFrame(training_org.loc[~training_org["windGust"].isnull(),"windSpeed"]),y=pd.DataFrame(training_org.loc[~training_org["windGust"].isnull(),"windGust"]))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [409]:
training_org.loc[training_org["windGust"].isnull(),"windGust"]=lr.predict(pd.DataFrame(training_org.loc[training_org["windGust"].isnull(),"windSpeed"]))

### cloudCover&uvIndex&pressure&ozone

In [410]:
training_org["cloudCover"]=training_org.groupby("summary")["cloudCover"].transform(lambda x:x.fillna(x.median()))

In [411]:
training_org["uvIndex"]=training_org.groupby("summary")["uvIndex"].transform(lambda x:x.fillna(x.median()))

In [412]:
training_org["pressure"]=training_org.groupby("summary")["pressure"].transform(lambda x:x.fillna(x.median()))

In [413]:
training_org["ozone"]=training_org.groupby("summary")["ozone"].transform(lambda x:x.fillna(x.median()))

In [414]:
training_org=training_org.dropna(subset=['ozone'])

### precipAccumulation-quit this variable

### precipAccumulation

In [415]:
## when precipAccumulation is null,weather is now
training_org.loc[training_org["icon"]=='snow','precipAccumulation'].isnull().sum()

0

In [416]:
test_org.loc[test_org["icon"]=='snow','precipAccumulation'].isnull().sum()

0

In [417]:
training_org['precipAccumulation']=training_org['precipAccumulation'].fillna(0)

In [418]:
test_org['precipAccumulation']=test_org['precipAccumulation'].fillna(0)

In [419]:
training_org.isnull().sum()

datetime               0
summary                0
icon                   0
precipIntensity        0
precipProbability      0
temperature            0
apparentTemperature    0
dewPoint               0
humidity               0
windSpeed              0
windGust               0
windBearing            0
cloudCover             0
uvIndex                0
visibility             0
precipType             0
pressure               0
precipAccumulation     0
ozone                  0
countRents             0
dtype: int64

In [420]:
len(training_org)

22416

In [421]:
test_org.isnull().sum()

datetime               0
summary                0
icon                   0
precipIntensity        0
precipProbability      0
temperature            0
apparentTemperature    0
dewPoint               0
humidity               0
windSpeed              0
windGust               0
windBearing            0
cloudCover             0
uvIndex                0
visibility             0
precipType             0
pressure               0
precipAccumulation     0
ozone                  0
dtype: int64

## transfrom date

In [422]:
def convert_datetime(df, col):
  df[col] = pd.to_datetime(df[col])
  df['day_of_week'] = df[col].dt.weekday_name 
  df['year'] = df[col].dt.year
  df['month'] = df[col].dt.month 
  df['day'] = df[col].dt.day 
  df['hour'] = df[col].dt.hour
  
  return None                         

convert_datetime(training_org, 'datetime')
convert_datetime(test_org, 'datetime')

## merge train and test with holiday

In [423]:
import datetime as dt

In [424]:
training_org["date2"]=pd.to_datetime(training_org["datetime"]).dt.strftime('%Y-%m-%d')

In [425]:
test_org["date2"]=pd.to_datetime(test_org["datetime"]).dt.strftime('%Y-%m-%d')

In [426]:
training_org=training_org.merge(
  holiday_org,
  left_on='date2',
  right_on='date',
  how="left"
)

In [427]:
test_org=test_org.merge(
  holiday_org,
  left_on='date2',
  right_on='date',
  how="left"
)

In [428]:
training_org['holiday']=training_org['holiday'].fillna("N")
test_org['holiday']=test_org['holiday'].fillna("N")

In [429]:
training_org.head()

,datetime,summary,icon,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,windSpeed,...,ozone,countRents,day_of_week,year,month,day,hour,date2,date,holiday
0,2017-01-01 00:00:00,Mostly Cloudy,partly-cloudy-night,0.0000,0.0,6.80,6.11,5.66,0.92,1.41,...,336.2,932,Sunday,2017,1,1,0,2017-01-01,NaN,N
1,2017-01-01 01:00:00,Mostly Cloudy,partly-cloudy-night,0.0000,0.0,6.36,5.28,5.27,0.93,1.69,...,336.2,892,Sunday,2017,1,1,1,2017-01-01,NaN,N
2,2017-01-01 02:00:00,Mostly Cloudy,partly-cloudy-night,0.0000,0.0,6.04,5.12,4.80,0.92,1.51,...,336.2,339,Sunday,2017,1,1,2,2017-01-01,NaN,N
3,2017-01-01 03:00:00,Mostly Cloudy,partly-cloudy-night,0.0000,0.0,5.79,5.79,4.61,0.92,1.33,...,336.2,282,Sunday,2017,1,1,3,2017-01-01,NaN,N
4,2017-01-01 04:00:00,Mostly Cloudy,partly-cloudy-night,0.0116,0.1,5.57,4.74,4.41,0.92,1.39,...,336.2,185,Sunday,2017,1,1,4,2017-01-01,NaN,N


In [430]:
test_org.head()

,datetime,summary,icon,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,windSpeed,...,precipAccumulation,ozone,day_of_week,year,month,day,hour,date2,date,holiday
0,2019-07-27 00:00:00,Mostly Cloudy,partly-cloudy-night,0.1727,0.31,20.19,20.47,17.45,0.84,4.03,...,0.0,303.8,Saturday,2019,7,27,0,2019-07-27,NaN,N
1,2019-07-27 01:00:00,Possible Drizzle,rain,0.2754,0.49,19.47,19.64,16.52,0.83,4.16,...,0.0,305.1,Saturday,2019,7,27,1,2019-07-27,NaN,N
2,2019-07-27 02:00:00,Possible Light Rain,rain,0.5042,0.62,18.40,18.57,16.17,0.87,3.81,...,0.0,306.6,Saturday,2019,7,27,2,2019-07-27,NaN,N
3,2019-07-27 03:00:00,Possible Light Rain,rain,0.7826,0.66,18.12,18.23,15.73,0.86,3.03,...,0.0,308.2,Saturday,2019,7,27,3,2019-07-27,NaN,N
4,2019-07-27 04:00:00,Possible Light Rain,rain,0.9662,0.69,17.69,17.87,16.04,0.90,2.93,...,0.0,310.2,Saturday,2019,7,27,4,2019-07-27,NaN,N


# First simple model

In [431]:
data_1=pd.concat([training_org,test_org],ignore_index=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [432]:
df_dummy_1= pd.get_dummies(data_1.drop(['date','datetime','date2'],axis=1).select_dtypes(include='object'))

In [433]:
data_1=data_1.merge(
  df_dummy_1,
  left_index=True, 
  right_index=True,
  how="left",
)
data_1.head()

,apparentTemperature,cloudCover,countRents,date,date2,datetime,day,day_of_week,dewPoint,holiday,...,summary_Possible Flurries,summary_Possible Light Rain,summary_Possible Light Rain and Windy,summary_Possible Light Sleet,summary_Possible Light Snow,summary_Rain,summary_Windy,summary_Windy and Mostly Cloudy,summary_Windy and Overcast,summary_Windy and Partly Cloudy
0,6.11,0.77,932.0,NaN,2017-01-01,2017-01-01 00:00:00,1,Sunday,5.66,N,...,0,0,0,0,0,0,0,0,0,0
1,5.28,0.77,892.0,NaN,2017-01-01,2017-01-01 01:00:00,1,Sunday,5.27,N,...,0,0,0,0,0,0,0,0,0,0
2,5.12,0.83,339.0,NaN,2017-01-01,2017-01-01 02:00:00,1,Sunday,4.80,N,...,0,0,0,0,0,0,0,0,0,0
3,5.79,0.71,282.0,NaN,2017-01-01,2017-01-01 03:00:00,1,Sunday,4.61,N,...,0,0,0,0,0,0,0,0,0,0
4,4.74,0.83,185.0,NaN,2017-01-01,2017-01-01 04:00:00,1,Sunday,4.41,N,...,0,0,0,0,0,0,0,0,0,0


In [434]:
data_1=data_1.drop(['date','datetime','date2'],axis=1).select_dtypes(exclude='object')

In [435]:
train_data_1=data_1[0:22416]
test_data_1=data_1[22416:]

In [436]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [437]:
X_train1, X_val1, y_train1, y_val1= train_test_split(train_data_1.drop(["countRents"],axis=1), train_data_1["countRents"], test_size=0.2, random_state=1)

In [438]:
model1 = LinearRegression().fit(X_train1, y_train1)

In [439]:
model1.score(X_train1, y_train1)

0.41347152573478835

In [440]:
model1.score(X_val1, y_val1)

0.40873645766975825

In [441]:
model1.score(train_data_1.drop(["countRents"],axis=1), train_data_1["countRents"])

0.4125418624495808

In [442]:
pred1=model1.predict(test_data_1.drop(["countRents"],axis=1))

In [445]:
sub1=pd.DataFrame()
sub1["datetime"]=test_org.datetime
sub1["countRents"]=pred1

In [446]:
sub1.head()

,datetime,countRents
0,2019-07-27 00:00:00,120.882033
1,2019-07-27 01:00:00,503.251099
2,2019-07-27 02:00:00,468.497385
3,2019-07-27 03:00:00,449.840777
4,2019-07-27 04:00:00,408.907870


In [447]:
sub1.to_csv('D:/CLASS/Term 2/Kaggle competition/submission1.csv')

## Model 2-random forest(my best model)

In [448]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [449]:
param_grid = {
    'bootstrap': [True],
    'n_estimators': [100, 200]
}

In [450]:
model5 = RandomForestRegressor()

In [451]:
grid_search = GridSearchCV(estimator = model5, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [452]:
grid_search.fit(train_data_1.drop(["countRents"],axis=1), train_data_1["countRents"])

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  2.4min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  2.9min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'bootstrap': [True], 'n_estimator

In [453]:
grid_search.best_params_

{'bootstrap': True, 'n_estimators': 200}

In [454]:
grid_search.score(train_data_1.drop(["countRents"],axis=1), train_data_1["countRents"])

0.996267663398425

In [456]:
pred2=grid_search.predict(test_data_1.drop(["countRents"],axis=1))

In [457]:
sub2=pd.DataFrame()
sub2["datetime"]=test_org.datetime
sub2["countRents"]=pred2

In [458]:
sub2.head()

,datetime,countRents
0,2019-07-27 00:00:00,446.670
1,2019-07-27 01:00:00,241.345
2,2019-07-27 02:00:00,162.455
3,2019-07-27 03:00:00,93.990
4,2019-07-27 04:00:00,99.085


In [459]:
sub2.to_csv('D:/CLASS/Term 2/Kaggle competition/submission2.csv')

# Feature engineering

In [461]:
from sklearn.preprocessing import StandardScaler

# Separating out the features
x = data_1.drop("countRents",axis=1)
# Separating out the target
y = data_1.loc[:,['countRents']]
# Standardizing the features
x = StandardScaler().fit_transform(x)

In [463]:
from sklearn.decomposition import PCA
pca = PCA(n_components=60) #selecting the number of components
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents,index=data_1.index)

In [465]:
# Concatenating the PCA table with SalePrice
data_2 = pd.concat([principalDf,data_1["countRents"]],axis=1,ignore_index=True)
data_2.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,-0.320610,2.416065,1.213855,-0.110627,1.335879,-0.605541,0.962802,0.100382,-1.302941,-1.184277,...,0.000186,-1.163947e-14,-1.542892e-15,-1.819965e-16,1.525250e-15,2.070778e-16,2.074893e-15,-6.365678e-16,6.786448e-16,932.0
1,-0.210394,2.614847,1.267246,-0.130423,1.304556,-0.575409,1.118705,0.057518,-1.146027,-1.126866,...,0.000334,-2.447250e-15,1.672089e-15,4.993871e-17,3.637427e-15,-1.969658e-15,-4.211689e-15,8.363267e-16,-7.957533e-16,892.0
2,-0.222023,2.611358,1.325544,-0.131734,1.362324,-0.613508,1.036879,0.085867,-1.198413,-1.088888,...,0.000123,1.238601e-14,-9.578073e-16,-7.367070e-16,-1.712887e-15,-1.168717e-15,2.483505e-16,-2.068568e-15,-7.638945e-16,339.0
3,-0.323405,2.648263,1.180280,-0.071300,1.362396,-0.595853,1.110504,0.078411,-1.160023,-1.051147,...,0.000589,-1.850525e-14,4.467108e-15,-4.766450e-17,2.197553e-15,3.326546e-15,-7.667379e-16,-1.614624e-16,-1.401976e-15,282.0
4,1.249731,2.068710,0.851041,-0.150964,1.219143,-0.560714,0.994243,-0.280646,-0.568108,-1.928774,...,0.000540,2.382342e-15,4.221643e-15,2.252910e-16,4.932037e-16,1.952772e-15,1.163907e-15,-1.147721e-16,-5.435985e-16,185.0


In [466]:
# Spilitting into Train 3 and Test 3 Set

train_data_2=data_2[0:22416]
test_data_2=data_2[22416:]

## Third model

In [468]:
from sklearn.ensemble import RandomForestRegressor

In [469]:
model3 = RandomForestRegressor(bootstrap=True,n_estimators=200)

In [476]:
model3.fit(train_data_2.iloc[:,:-1], train_data_2.iloc[:,-1])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [477]:
model3.score(train_data_2.iloc[:,:-1], train_data_2.iloc[:,-1])

0.9499187126165082

In [479]:
pred3=model3.predict(test_data_2.iloc[:,:-1])

In [481]:
sub3=pd.DataFrame()
sub3["datetime"]=test_org.datetime
sub3["countRents"]=pred3

In [482]:
sub3.head()

,datetime,countRents
0,2019-07-27 00:00:00,337.250
1,2019-07-27 01:00:00,1833.900
2,2019-07-27 02:00:00,843.275
3,2019-07-27 03:00:00,362.385
4,2019-07-27 04:00:00,408.440


In [483]:
sub3.to_csv('D:/CLASS/Term 2/Kaggle competition/submission3.csv')

## Fourth model

In [484]:
import xgboost

In [487]:
model4=xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=1000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 

In [488]:
model4.fit(train_data_1.drop(["countRents"],axis=1), train_data_1["countRents"])

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[12:32:46] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4, gamma=0,
             importance_type='gain', learning_rate=0.07, max_delta_step=0,
             max_depth=3, min_child_weight=1.5, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1, seed=42,
             silent=None, subsample=0.6, verbosity=1)

In [489]:
model4.score(train_data_1.drop(["countRents"],axis=1), train_data_1["countRents"])

0.9349381942400025

In [491]:
pred4=model4.predict(test_data_1.drop(["countRents"],axis=1))

In [492]:
sub4=pd.DataFrame()
sub4["datetime"]=test_org.datetime
sub4["countRents"]=pred4

In [493]:
sub4.to_csv('D:/CLASS/Term 2/Kaggle competition/submission4.csv')